Go to https://drive.google.com/drive/folders/1Pe6D713L9S0GWwb_XzeLXAeNZOrBqZaN?usp=sharing and click add shortcut to drive. This will add the data required for this assignment to your Google drive.

<img src="https://drive.google.com/uc?id=1LqHisiziX8Ri94Xs6Cv8mhx6vivFM3kS" alt="Drawing" height="300"/>

Caution: Since this is real language on Twitter and deals with world events, some of it could be disturbing. In the later section of the course, we will deal with ethics and what is appropriate and what is not. 

First, let's read the tweets from the Google drive and print a few lines from the file.



Run the below code snippet. It will generate a URL which generates an authorization code. Enter it below to give Colab access to your Google drive. 

**Note:** Copy button may not work. Try copying the authorization code manually.

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


When you run the `ls` command below, you should see the files in the tweets folder.




In [3]:
!ls "/content/drive/My Drive/projects/phd/courses/596/tweets"

20000_tweets.jsonl
20000_tweets.txt
covid-tweets-2020-08-10-2020-08-21.tokenized.txt
covid-tweets-2020-08-10-2020-08-21.trigrams.txt
GoogleNews-vectors-negative300.bin.gz
stop_words.txt


Let's read the top 10 tweets from the file `20000_tweets.txt` and print them.

In [4]:
f = open("/content/drive/My Drive/projects/phd/courses/596/tweets/20000_tweets.txt", "r")

line_count = 0
top_tweets = []
for tweet in f:
  print("### Tweet", line_count, "#####")
  print(tweet)
  
  top_tweets.append(tweet)
  line_count += 1
  if line_count >= 10:
    break
f.close()

### Tweet 0 #####
Covid-19 Economic Response: Cancel Student Loans by Executive Order. - Sign the Petition! https://t.co/BnPXWHv5cr via @Change

### Tweet 1 #####
Hey! The stock markets up! Fuck Trump and the Trumpublican Senate! https://t.co/4t6mgbaG2C

### Tweet 2 #####
@ProjectLincoln blame Trump for no sports because of Covid19 @MeidasTouch

### Tweet 3 #####
Do u guys think that reopening of the dinning in the restaurants of Lahore will bring any change in our lives? Nothing is going to change I think cause many people have learned cooking, painting and many new stuff during lockdown and I think we all will continue those things 1/2

### Tweet 4 #####
@NBCSAthletics Ya just knew the season wouldn’t go by without some bench clearing brawls... Covid or not. Behaviors can’t be changed because of rules, sadly

### Tweet 5 #####
You’re comparing apples and oranges. The NCAA has a lot more to balance than the NFL. https://t.co/oeMsbgeem7

### Tweet 6 #####
THANK YOU @POTUS Absolutely NO

# **Problem 1: Word Tokenizer**

Implement your own tokenizer for tweets without using libraries like `nltk`, `spacy` or any existing tokenizers. Tokenizer is a fundamental tool for processing any text data. The tokenizer should preserve punctuations.

The outputs for the top five tweets should approximately be as follows (it's fine to not exactly match this output)

```
['Covid-19', 'Economic', 'Response', ':', 'Cancel', 'Student', 'Loans', 'by', 'Executive', 'Order', '.', '-', 'Sign', 'the', 'Petition', '!', 'https://t.co/BnPXWHv5cr', 'via', '@Change']
['Hey', '!', 'The', 'stock', 'markets', 'up', '!', 'Fuck', 'Trump', 'and', 'the', 'Trumpublican', 'Senate', '!', 'https://t.co/4t6mgbaG2C']
['@ProjectLincoln', 'blame', 'Trump', 'for', 'no', 'sports', 'because', 'of', 'Covid19', '@MeidasTouch']
['Do', 'u', 'guys', 'think', 'that', 'reopening', 'of', 'the', 'dinning', 'in', 'the', 'restaurants', 'of', 'Lahore', 'will', 'bring', 'any', 'change', 'in', 'our', 'lives', '?', 'Nothing', 'is', 'going', 'to', 'change', 'I', 'think', 'cause', 'many', 'people', 'have', 'learned', 'cooking', ',', 'painting', 'and', 'many', 'new', 'stuff', 'during', 'lockdown', 'and', 'I', 'think', 'we', 'all', 'will', 'continue', 'those', 'things', '1/2']
['@NBCSAthletics', 'Ya', 'just', 'knew', 'the', 'season', 'would', 'n’t', 'go', 'by', 'without', 'some', 'bench', 'clearing', 'brawls', '...', 'Covid', 'or', 'not', '.', 'Behaviors', 'ca', 'n’t', 'be', 'changed', 'because', 'of', 'rules', ',', 'sadly']
```

A thing to note is that words like `wouldn't` and `you'll` become two tokens `would n't` and `you 'll`, after tokenization. Treating `'nt` and  `'ll` as independent tokens is common in NLP tools. You can come up with that kind of list easily, e.g., `'s, 've`.


In [25]:
# import any relevant libraries here
import re

# a function to tokenize text into words
def tokenize(text):
    # FILL ME

    # names with two words to keep as a single token
    names = [ "New Zealand" ]
    # single words that should be split into two tokens
    concats = [ "n't", "'ll", "'s", "'ve", "'re", "n’t", "’ll", "’s", "’ve", "’re" ]
    # punctuation after word that should be split out
    ending_puncs = [ "!", "?", ",", "...", ".", ":", ";" ]
    # punctuation in middle of chars that should result in split
    middle_puncs_search_regexes = [ "[a-zA-Z]+/[a-zA-Z]+", ".+\.\.\..+" ]
    middle_puncs_split_regexes = [ "(/)", "(\.\.\.)" ]

    # strip trailing whitespace and newlines
    text = text.rstrip()

    # text we have left to tokenize
    text_left = text

    # tokens created from this text
    words = []

    # iterate until we have no more raw text to tokenize
    while len(text_left) != 0:

        # flag to move onto next iteration of processing
        next_iter = False

        # first strip leading whitespace
        text_left = text_left.lstrip()

        # second check for proper names containing spaces
        for name in names:
            if text_left.startswith(name):
                words.append(name) # add name as token
                text_left = text_left[len(name):] # cut out processed token from unprocessed text
                next_iter = True # indicate to move on to next processing iteration
                break

        # check if we should move on to next iteration
        if next_iter:
            continue

        # now can safely just refine to looking at characters up to next space
        match = re.match("[^ ]+ ", text_left)
        
        if match:
            # if there's a match, strip the trailing whitespace off
            no_space_text = match.group().rstrip()
            text_left = text_left[len(no_space_text):]
        else:
            # if there's no match, it means we reached the last word of the remaining text
            no_space_text = text_left
            text_left = ""
        
        # look for urls
        match = re.match("https://[a-zA-Z0-9]+\.[a-zA-Z]{2,3}/.*", no_space_text)

        if match:
            # word we're looking at is a URL, don't split on non-alphanumeric chars
            words.append(match.group())
            continue

        # from this point there could be multiple tokens in no_space_text

        # check for ending punctuation which we need to split off
        for ending_punc in ending_puncs:
            if no_space_text.endswith(ending_punc):
                word_and_punc = [ no_space_text[:-len(ending_punc)], ending_punc ]
                break
        else:
            word_and_punc = [ no_space_text ]

        # check for middle punctuation that means we need to break up word
        divided_words = []
        for idx in range(len(middle_puncs_search_regexes)):
            m = re.search(middle_puncs_search_regexes[idx], word_and_punc[0])
            if m is not None:
                divided_words = re.split(middle_puncs_split_regexes[idx], word_and_punc[0])
                break
        else:
            divided_words = [ word_and_punc[0] ]

        # add ending punc back on
        if len(word_and_punc) == 2:
            divided_words.append(word_and_punc[1])

        # now lets check for concatenated words
        new_words = []
        for word in divided_words:
            for concat in concats:
                if word.endswith(concat):
                    # add correctly split words to tokens
                    new_words.append(word[:-len(concat)])
                    new_words.append(concat)
                    break
            else:
                new_words.append(word)

        # add words to all tokens
        words += new_words

    return words

tokenized_top_tweets = [tokenize(tweet) for tweet in top_tweets]
for tokenized_tweet in tokenized_top_tweets:
  print(tokenized_tweet)

['Covid-19', 'Economic', 'Response', ':', 'Cancel', 'Student', 'Loans', 'by', 'Executive', 'Order', '.', '-', 'Sign', 'the', 'Petition', '!', 'https://t.co/BnPXWHv5cr', 'via', '@Change']
['Hey', '!', 'The', 'stock', 'markets', 'up', '!', 'Fuck', 'Trump', 'and', 'the', 'Trumpublican', 'Senate', '!', 'https://t.co/4t6mgbaG2C']
['@ProjectLincoln', 'blame', 'Trump', 'for', 'no', 'sports', 'because', 'of', 'Covid19', '@MeidasTouch']
['Do', 'u', 'guys', 'think', 'that', 'reopening', 'of', 'the', 'dinning', 'in', 'the', 'restaurants', 'of', 'Lahore', 'will', 'bring', 'any', 'change', 'in', 'our', 'lives', '?', 'Nothing', 'is', 'going', 'to', 'change', 'I', 'think', 'cause', 'many', 'people', 'have', 'learned', 'cooking', ',', 'painting', 'and', 'many', 'new', 'stuff', 'during', 'lockdown', 'and', 'I', 'think', 'we', 'all', 'will', 'continue', 'those', 'things', '1/2']
['@NBCSAthletics', 'Ya', 'just', 'knew', 'the', 'season', 'would', 'n’t', 'go', 'by', 'without', 'some', 'bench', 'clearing', 

# Problem 2: Post-process the tokenized tweets

Clean the tokenized tweets such that usernames are repalced with `@USER`, URLs are replaced with `URL`, punctuations are removed and words are lower-cased.

The output for the top ten tweets after cleaning should look something like this

```
['covid-19', 'economic', 'response', 'cancel', 'student', 'loans', 'by', 'executive', 'order', 'sign', 'the', 'petition', 'URL', 'via', '@USER']
['hey', 'the', 'stock', 'markets', 'up', 'fuck', 'trump', 'and', 'the', 'trumpublican', 'senate', 'URL']
['@USER', 'blame', 'trump', 'for', 'no', 'sports', 'because', 'of', 'covid19', '@USER']
['do', 'u', 'guys', 'think', 'that', 'reopening', 'of', 'the', 'dinning', 'in', 'the', 'restaurants', 'of', 'lahore', 'will', 'bring', 'any', 'change', 'in', 'our', 'lives', 'nothing', 'is', 'going', 'to', 'change', 'i', 'think', 'cause', 'many', 'people', 'have', 'learned', 'cooking', 'painting', 'and', 'many', 'new', 'stuff', 'during', 'lockdown', 'and', 'i', 'think', 'we', 'all', 'will', 'continue', 'those', 'things', '1/2']
['@USER', 'ya', 'just', 'knew', 'the', 'season', 'would', 'n’t', 'go', 'by', 'without', 'some', 'bench', 'clearing', 'brawls', 'covid', 'or', 'not', 'behaviors', 'ca', 'n’t', 'be', 'changed', 'because', 'of', 'rules', 'sadly']
```

In [31]:
# import relevant packages

# function to clean a tweet
def clean_a_tweet(tokenized_tweet):
  
    clean_tokenized_tweet = []

    for token in tokenized_tweet:
        # deal with users
        if token.startswith("@"):
            clean_tokenized_tweet.append("@USER")
        # deal with URLS
        elif re.match("https://[a-zA-Z0-9]+\.[a-zA-Z]{2,3}/.*", token) is not None:
            clean_tokenized_tweet.append("URL")
        # deal with punctuation
        elif token in ["/", "...", "?", "!", ".", ",", ":", ";", "-", "%"]:
            continue
        # otherwise just lowercase the word
        else:
            clean_tokenized_tweet.append(token.lower())


    return(clean_tokenized_tweet)

anonymized_top_tweets = [clean_a_tweet(tokenized_tweet) for tokenized_tweet in tokenized_top_tweets]
for tokenized_tweet in anonymized_top_tweets:
  print(tokenized_tweet)

['covid-19', 'economic', 'response', 'cancel', 'student', 'loans', 'by', 'executive', 'order', 'sign', 'the', 'petition', 'URL', 'via', '@USER']
['hey', 'the', 'stock', 'markets', 'up', 'fuck', 'trump', 'and', 'the', 'trumpublican', 'senate', 'URL']
['@USER', 'blame', 'trump', 'for', 'no', 'sports', 'because', 'of', 'covid19', '@USER']
['do', 'u', 'guys', 'think', 'that', 'reopening', 'of', 'the', 'dinning', 'in', 'the', 'restaurants', 'of', 'lahore', 'will', 'bring', 'any', 'change', 'in', 'our', 'lives', 'nothing', 'is', 'going', 'to', 'change', 'i', 'think', 'cause', 'many', 'people', 'have', 'learned', 'cooking', 'painting', 'and', 'many', 'new', 'stuff', 'during', 'lockdown', 'and', 'i', 'think', 'we', 'all', 'will', 'continue', 'those', 'things', '1/2']
['@USER', 'ya', 'just', 'knew', 'the', 'season', 'would', 'n’t', 'go', 'by', 'without', 'some', 'bench', 'clearing', 'brawls', 'covid', 'or', 'not', 'behaviors', 'ca', 'n’t', 'be', 'changed', 'because', 'of', 'rules', 'sadly']
['y

# Problem 3: Unigrams and Bigrams

Read **all** tweets from the file `20000_tweets.txt` and print the top 10 unigrams and bigrams. Consider only the English tweets. You have to use `tokenize` and `clean_a_tweet` function.

Output format

```
Top 10 Unigrams
@USER	16698
the	16666
URL	12610
to	11866
a	9845
....
....
....
....
....

Top 10 Bigrams
@USER @USER	7494
in the	1280
of the	1265
it ’s	860
a mask	793
....
....
....
....
....
```

Your numbers need not match the above word frequencies. Depending on your tokenizer, you may get different numbers

In [34]:
# Write your code to build top 10 unigrams and bigrams
# Hint: You can use python dictionary or collections.Counter().

from collections import Counter

unigrams = Counter()
bigrams = Counter()

with open("/content/drive/My Drive/projects/phd/courses/596/tweets/20000_tweets.txt", "r") as f:

    tweets = []
    for tweet in f:
        tokens = tokenize(tweet)
        clean_tokens = clean_a_tweet(tokens)
        # build up unigram count
        unigrams.update(clean_tokens)

        # build up bigram count
        for idx in range(len(clean_tokens) - 1):
            bigram = ( clean_tokens[idx], clean_tokens[idx + 1] )
            bigrams[bigram] += 1

# sort unigrams and bigrams
top_unigrams = unigrams.most_common(10)
top_bigrams = bigrams.most_common(10)

print("Top 10 Unigrams")
for top_unigram in top_unigrams:
    print(f"{top_unigram[0]}    {top_unigram[1]}")

print("")
print("Top 10 Bigrams")
for top_bigram in top_bigrams:
    print(f"{top_bigram[0][0]} {top_bigram[0][1]}    {top_bigram[1]}")


Top 10 Unigrams
@USER    16678
the    16537
URL    12576
to    11837
a    9775
and    8463
of    7804
is    6445
in    6260
i    5497

Top 10 Bigrams
@USER @USER    7489
in the    1273
of the    1262
it ’s    837
a mask    749
this is    722
the pandemic    696
wear a    688
URL URL    682
to be    660


# Problem 4: How frequent are the following unigrams and bigrams?

```
covid
coronavirus
republicans
democrats
social distancing
wear mask
no mask
donald trump
joe biden
```

Output:
```
covid	4437
coronavirus	1368
...
...
social distancing	568
wear mask	19
...
...
...
```


In [36]:
# Write your code to print the frequencies of the above unigrams and bigrams
# Hint: reuse the dictionaries you built in the previous cell

unigrams_of_interest = [ "covid", "coronavirus", "republicans", "democrats" ]
bigrams_of_interest = [ ( "social", "distancing" ), ( "wear", "mask" ), ( "no", "mask" ), ( "donald", "trump" ), ( "joe", "biden" ) ]

for unigram_of_interest in unigrams_of_interest:
    print(f"{unigram_of_interest}   {unigrams[unigram_of_interest]}")

for bigram_of_interest in bigrams_of_interest:
    print(f"{bigram_of_interest[0]} {bigram_of_interest[1]}    {bigrams[bigram_of_interest]}")

covid   4188
coronavirus   1327
republicans   56
democrats   158
social distancing    532
wear mask    19
no mask    19
donald trump    61
joe biden    19
